In [ ]:
from QAOA_simulator import get_simulator, get_expectation, get_result
from plot_utils import plot_heat_map
import networkx as nx
import qokit.maxcut as mc
import numpy as np
from QAOA_proxy import QAOA_proxy
from QAOA_paper_proxy import QAOA_paper_proxy


def collect_parameter_data(G: nx.Graph, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])
    N = G.number_of_nodes()
    ising_model = mc.get_maxcut_terms(G)
    sim = get_simulator(N, ising_model)

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            result = get_result(N, ising_model, gamma, beta, sim)
            expectations[i][j] = get_expectation(N, ising_model, gamma, beta, sim, result)

    return expectations


def collect_parameter_data_proxy_less_dumb(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


def collect_parameter_data_proxy(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations


N = 6
G = nx.erdos_renyi_graph(N, 0.5, seed=18)  # has 10 edges
gammas = np.linspace(-np.pi, np.pi, 50)
betas = np.linspace(-2 * np.pi, 2 * np.pi, 50)
expectations = collect_parameter_data(G, gammas, betas)
_ = plot_heat_map(gammas, betas, expectations, "True QAOA Expectation", "Gamma", "Beta")
expectation_proxies = collect_parameter_data_proxy(G.number_of_edges(), N, gammas, betas)
_ = plot_heat_map(gammas, betas, expectation_proxies, "Expectation Proxies from Paper", "Gamma", "Beta")
expectation_proxies_less_dumb = collect_parameter_data_proxy_less_dumb(G.number_of_edges(), N, gammas, betas)
_ = plot_heat_map(gammas, betas, expectation_proxies_less_dumb, "Expectation Proxies from us", "Gamma", "Beta")

In [ ]:
from QAOA_paper_proxy import QAOA_paper_proxy
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np

"""
Paper proxy only
"""

def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_paper_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations

gammas = np.linspace(-np.pi, np.pi, 50)
betas = np.linspace(-2 * np.pi, 2 * np.pi, 50)

# Run experiments for each N
for n in range(2, 16):
    m = int(n * (n - 1) / 4)
    if m < n:
        m = n
    expectations = collect_parameter_data(m, n, gammas, betas)
    plot_heat_map(gammas, betas, expectations, f"Expectation Proxy from paper (N={n} M={m})", "Gamma", "Beta")
    plt.show()

    # Save data
    np.save(f"data_for_Expectation_Heatmap/Paper_Proxy_N={n}_M={m}.npy", expectations)

In [ ]:
from QAOA_proxy import QAOA_proxy, QAOA_proxy_run
from plot_utils import plot_heat_map
import matplotlib.pyplot as plt
import numpy as np

"""
Our proxy only
"""

def collect_parameter_data(num_constraints: int, num_qubits: int, gammas: np.ndarray, betas: np.ndarray) -> np:
    expectations = np.zeros([len(gammas), len(betas)])

    for i in range(len(gammas)):
        for j in range(len(betas)):
            gamma = np.array([gammas[i]])
            beta = np.array([betas[j]])
            _, expectations[i][j] = QAOA_proxy(1, gamma, beta, num_constraints, num_qubits)

    return expectations

gammas = np.linspace(-np.pi, np.pi, 50)
betas = np.linspace(-2 * np.pi, 2 * np.pi, 50)

# Run experiments for each N
for n in range(5, 16):
    m = int(n * (n - 1) / 4)
    if m < n:
        m = n
    expectations = collect_parameter_data(m, n, gammas, betas)
    plot_heat_map(gammas, betas, expectations, f"Expectation Proxy from us (N={n} M={m})", "Gamma", "Beta")
    plt.show()

    result = QAOA_proxy_run(m, n, 1, np.array([0.01]), np.array([0.01]))
    print(result)

    # Save data
    np.save(f"data_for_Expectation_Heatmap/Our_Proxy_N={n}_M={m}.npy", expectations)
